# Summarization

In [ ]:
from datasets import load_dataset

# billsum = load_dataset("xsum", split="train[:5%]").filter(lambda x: len(x["document"]) <= 1000)
# billsumTest = load_dataset("xsum", split="test[:5%]").filter(lambda x: len(x["document"]) <= 1000)
billsumEval = load_dataset("xsum", split="validation[:5%]").filter(lambda x: len(x["document"]) <= 1000)

In [1]:
from transformers import DataCollatorForLanguageModeling, GPT2LMHeadModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, TrainingArguments, Trainer
from transformers import GPT2Tokenizer, GPT2Config
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", max_length=1000)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("outputdir")

C:\Users\GIGABYTE\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model.loss

AttributeError: 'GPT2LMHeadModel' object has no attribute 'loss'

In [ ]:
def generate_text(sequence, max_new_tokens = 50, min_new_tokens = 10, temperature = 1, top_k = 50, top_p = 1.0, rep_penalty = 1.0, model_trained = True):
    if model_trained:
        model = GPT2LMHeadModel.from_pretrained("outputdir")
    else:
        model = GPT2LMHeadModel.from_pretrained("gpt2")
        
    text = "\n\nSummarize: " + sequence + "\n\n"
    input = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    output = model.generate(input["input_ids"], attention_mask=input["attention_mask"], max_new_tokens = max_new_tokens, do_sample=True, min_new_tokens=min_new_tokens, temperature = temperature, top_k = top_k, top_p = top_p, repetition_penalty = rep_penalty, pad_token_id=tokenizer.eos_token_id)[0]

    return  tokenizer.decode(output)[len(text):]


In [ ]:
import evaluate
rouge = evaluate.load("rouge")

In [ ]:
rouge.compute(predictions=["Hello"],references=["Hello"])['rouge1']

In [ ]:
WynikiGPT2U_temp = []
WynikiGPT2_temp = []
for temp in range(1, 21):
    sumModel = 0   
    sumModelGPT2 = 0
    for i in range(1,6):
        print("(TEMP: " + str(temp/10) + ", ID: " + str(i) + ")")
        sumModel = sumModel + rouge.compute(predictions=[generate_text(billsumEval[i]["document"], temperature = temp/10, top_k = 30)],references=[billsumEval[i]["summary"]])['rougeL']
        sumModelGPT2 = sumModelGPT2 + (rouge.compute(predictions=[generate_text(billsumEval[i]["document"], temperature = temp/10, top_k = 30, model_trained = False)],references=[billsumEval[i]["summary"]]))['rougeL']
    WynikiGPT2U_temp.append(sumModel/5)
    WynikiGPT2_temp.append(sumModelGPT2/5)

In [ ]:
WynikiGPT2U_topP = []
WynikiGPT2_topP = []
for top_p in range(1, 11):
    sumModel = 0   
    sumModelGPT2 = 0
    for i in range(1,6):
        print("(topP: " + str(top_p/10) + ", ID: " + str(i) + ")")
        sumModel = sumModel + rouge.compute(predictions=[generate_text(billsumEval[i]["document"], top_p = top_p/10)],references=[billsumEval[i]["summary"]])['rougeL']
        sumModelGPT2 = sumModelGPT2 + (rouge.compute(predictions=[generate_text(billsumEval[i]["document"], top_p = top_p/10, model_trained = False)],references=[billsumEval[i]["summary"]]))['rougeL']
    WynikiGPT2U_topP.append(sumModel/5)
    WynikiGPT2_topP.append(sumModelGPT2/5)


In [ ]:
WynikiGPT2U_topK = []
WynikiGPT2_topK = []
for topK in range(2, 50, 2):
    sumModel = 0   
    sumModelGPT2 = 0
    for i in range(1,6):
        print("(topK: " + str(topK) + ", ID: " + str(i) + ")")
        sumModel = sumModel + rouge.compute(predictions=[generate_text(billsumEval[i]["document"], top_k = topK)],references=[billsumEval[i]["summary"]])['rougeL']
        sumModelGPT2 = sumModelGPT2 + (rouge.compute(predictions=[generate_text(billsumEval[i]["document"], top_k = topK, model_trained = False)],references=[billsumEval[i]["summary"]]))['rougeL']
    WynikiGPT2U_topK.append(sumModel/5)
    WynikiGPT2_topK.append(sumModelGPT2/5)


In [ ]:
WynikiGPT2U_repPenalty = []
WynikiGPT2_repPenalty = []
for repPenalty in range(1, 21):
    sumModel = 0   
    sumModelGPT2 = 0
    for i in range(1,6):
        print("(repPenalty: " + str(repPenalty/10) + ", ID: " + str(i) + ")")
        sumModel = sumModel + rouge.compute(predictions=[generate_text(billsumEval[i]["document"], rep_penalty = repPenalty/10)],references=[billsumEval[i]["summary"]])['rougeL']
        sumModelGPT2 = sumModelGPT2 + (rouge.compute(predictions=[generate_text(billsumEval[i]["document"], rep_penalty = repPenalty/10, model_trained = False)],references=[billsumEval[i]["summary"]]))['rougeL']
    WynikiGPT2U_repPenalty.append(sumModel/5)
    WynikiGPT2_repPenalty.append(sumModelGPT2/5)


In [ ]:
import matplotlib.pyplot as plt
# TEMP

# Tworzenie wykresu
plt.plot([x/10 for x in list(range(1, 21))], WynikiGPT2U_temp, label='Trained', color='green')
plt.plot([x/10 for x in list(range(1, 21))], WynikiGPT2_temp, label='Basic')


# Dodanie poziomej linii na wartości y=7
plt.axhline(y=1, color='r', linestyle='-', label='Optymalny wynik')

# Dodanie legendy
plt.legend()
plt.title('Funkcja wyniku ROUGE-L')
plt.xlabel('temperature')
plt.ylabel('ROUGE-L')

# Wyświetlenie wykresu
plt.show()

#TOP_P

# Tworzenie wykresu
plt.plot([x/10 for x in list(range(1, 11))], WynikiGPT2U_topP, label='Trained', color='green')
plt.plot([x/10 for x in list(range(1, 11))], WynikiGPT2_topP, label='Basic')


# Dodanie poziomej linii na wartości y=7
plt.axhline(y=1, color='r', linestyle='-', label='Optymalny wynik')

# Dodanie legendy
plt.legend()
plt.title('Funkcja wyniku ROUGE-L')
plt.xlabel('TOP_P')
plt.ylabel('ROUGE-L')

# Wyświetlenie wykresu
plt.show()

# TOP_k

# Tworzenie wykresu
plt.plot([x for x in list(range(2, 50, 2))], WynikiGPT2U_topK, label='Trained', color='green')
plt.plot([x for x in list(range(2, 50, 2))], WynikiGPT2_topK, label='Basic')


# Dodanie poziomej linii na wartości y=7
plt.axhline(y=1, color='r', linestyle='-', label='Optymalny wynik')

# Dodanie legendy
plt.legend()
plt.title('Funkcja wyniku ROUGE-L')
plt.xlabel('TOP_k')
plt.ylabel('ROUGE-L')

# Wyświetlenie wykresu
plt.show()

# Repetition Penalty

# Tworzenie wykresu
plt.plot([x/10 for x in list(range(1, 21))], WynikiGPT2U_repPenalty, label='Trained', color='green')
plt.plot([x/10 for x in list(range(1, 21))], WynikiGPT2_repPenalty, label='Basic')


# Dodanie poziomej linii na wartości y=7
plt.axhline(y=1, color='r', linestyle='-', label='Optymalny wynik')

# Dodanie legendy
plt.legend()
plt.title('Funkcja wyniku ROUGE-L')
plt.xlabel('Repetition Penalty')
plt.ylabel('ROUGE-L')

# Wyświetlenie wykresu
plt.show()